In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

ROOT = '/content/drive/My Drive/FSL_project/'

In [3]:
X_train = pickle.load(open(ROOT + '/cifar/X_train', mode='rb'))
print(X_train.shape)
X_test = pickle.load(open(ROOT + '/cifar/X_test', mode='rb'))
print(X_test.shape)
X_val = pickle.load(open(ROOT + '/cifar/X_val', mode='rb'))
print(X_val.shape)
y_train = pickle.load(open(ROOT + '/cifar/y_train', mode='rb'))
print(y_train.shape)
y_test = pickle.load(open(ROOT + '/cifar/y_test', mode='rb'))
print(y_test.shape)
y_val = pickle.load(open(ROOT + '/cifar/y_val', mode='rb'))
print(y_val.shape)

(32000, 32, 32, 3)
(10000, 32, 32, 3)
(8000, 32, 32, 3)
(32000, 10)
(10000, 10)
(8000, 10)


In [4]:

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.datasets import cifar10
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

Using TensorFlow backend.


In [5]:
# Define the model
num_filters = 32

model = Sequential()
model.add(Conv2D(num_filters, (3,3), padding='same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Summarize the model
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         


In [0]:
import numpy as np
from tensorflow.python.keras.backend import image_data_format


class CutMixGenerator:
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, num_unlabled=28,
                 shuffle=True, data_gen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.data_gen = data_gen
        self.num_unlabled = num_unlabled

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size))  # 32000 // 128 = 250

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size:(i + 1) * self.batch_size] # [0:128], [128:256]
                X, y = self.__data_generation(batch_ids)
                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        _, class_num = self.y_train.shape

        X_labeled = self.X_train[batch_ids[:(self.batch_size - self.num_unlabled)]]
        X_unlabeled = self.X_train[batch_ids[(self.batch_size - self.num_unlabled):]] # batch_ids [100: 128]
      


        batch_index = batch_ids[(self.batch_size - self.num_unlabled):] 
        rand_index = np.random.permutation(batch_index)
        

        X1 = self.X_train[batch_index]
        # print("X1 :", X1.shape)
        X2 = self.X_train[rand_index]
        # print("X2 :", X2.shape)

        y_labeled = self.y_train[batch_ids[:(self.batch_size - self.num_unlabled)]]
        y_unlabeled = self.y_train[batch_ids[(self.batch_size - self.num_unlabled):]] # batch_ids [100: 128]


        y1 = self.y_train[batch_index]
        y2 = self.y_train[rand_index]
        lam = np.random.beta(self.alpha, self.alpha)


        bx1, by1, bx2, by2 = self.get_rand_bbox(w, h, lam)
        X1[:, bx1:bx2, by1:by2, :] = X2[:, bx1:bx2, by1:by2, :]

        X = np.concatenate((X_labeled, X1), axis=0)
        # print("X :", X.shape)

        y_1 = y1 * lam + y2 * (1 - lam)
        y = []
        y.append(np.concatenate((y_labeled, y_1), axis=0))

        if self.data_gen:
            for i in range(self.batch_size):
                X[i] = self.data_gen.random_transform(X[i])

        return X, y

    def get_rand_bbox(self, width, height, l):

      r_x = np.random.randint(width)
      r_y = np.random.randint(height)
      r_l = np.sqrt(1 - l)
      r_w = np.int(width * r_l)
      r_h = np.int(height * r_l)
      bb_x_1 = np.clip(r_x - r_w // 2, 0, width)
      bb_y_1 = np.clip(r_y - r_h // 2, 0, height)
      bb_x_2 = np.clip(r_x + r_w // 2, 0, width)
      bb_y_2 = np.clip(r_y + r_h // 2, 0, height)
      return bb_x_1, bb_y_1, bb_x_2, bb_y_2





In [11]:

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os


training_generator = CutMixGenerator(X_train, y_train, batch_size=128, alpha=0.2, num_unlabled = 28)()
model.fit_generator(generator=training_generator,
                    steps_per_epoch=250,
                    validation_data=(X_test, y_test),
                    epochs=100, verbose=1)

Epoch 1/100
250/250 [==============================] - 8s 32ms/step - loss: 1.4182 - acc: 0.5052 - val_loss: 1.2287 - val_acc: 0.5696
Epoch 2/100
250/250 [==============================] - 8s 31ms/step - loss: 1.2823 - acc: 0.5639 - val_loss: 1.0606 - val_acc: 0.6273
Epoch 3/100
250/250 [==============================] - 8s 31ms/step - loss: 1.1483 - acc: 0.6098 - val_loss: 0.9523 - val_acc: 0.6739
Epoch 4/100
250/250 [==============================] - 8s 31ms/step - loss: 1.0750 - acc: 0.6419 - val_loss: 0.9077 - val_acc: 0.6869
Epoch 5/100
250/250 [==============================] - 8s 31ms/step - loss: 1.0251 - acc: 0.6613 - val_loss: 0.8524 - val_acc: 0.7108
Epoch 6/100
250/250 [==============================] - 8s 31ms/step - loss: 0.9705 - acc: 0.6798 - val_loss: 0.8595 - val_acc: 0.7019
Epoch 7/100
250/250 [==============================] - 8s 31ms/step - loss: 0.9182 - acc: 0.7007 - val_loss: 0.8345 - val_acc: 0.7123
Epoch 8/100
250/250 [==============================] - 8s 31ms